# 基于streamlit的大模型应用开发3

In [1]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = ""

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
model_ty = Tongyi(temperature=0)
model_qf = QianfanLLMEndpoint(model="ERNIE-Bot")
chat_qf = QianfanChatEndpoint(model="ERNIE-Bot")
chat_xh = ChatSparkLLM()
chat_ty= ChatTongyi()

In [3]:
import langchain
langchain.__version__

'0.1.16'

In [5]:
#! pip install --upgrade langchain -i https://mirrors.aliyun.com/pypi/simple

## streamlit 的文件传入

https://docs.streamlit.io/develop/api-reference/widgets/st.file_uploader

https://github.com/langchain-ai/streamlit-agent/blob/main/streamlit_agent/chat_with_documents.py

* 如何获取file_uploader的文件目录

## streamlit的缓存

https://docs.streamlit.io/develop/api-reference/caching-and-state/st.cache_resource

https://zgpeace.blog.csdn.net/article/details/135665519

https://zhuanlan.zhihu.com/p/614509767?eqid=8f94f74e0004c61900000004649d4c19

https://docs.streamlit.io/develop/concepts/architecture/caching

* 数据缓存 st.cache_data

* 对象缓存 st.cache_resource


缓存：cache
缓存主要用来解决两个问题：

长时间运行的函数重复运行，这会减慢应用程序。
对象被重复创建，这使得它们很难在重新运行或会话中持久化。
在老版本的Streamlit中，缓存均通过装饰器st.cache来实现。
 在新版本中，缓存分成了两个装饰器st.cache_data和st.cache_resource

缓存数据：cache_data
cache_data适合返回DataFrames、NumPy 数组、str、int、float或者其他可序列化类型的函数。

缓存资源：cache_resource
缓存资源通常作用于缓存数据库连接和 ML 模型这类全局可用的资源。

当函数的返回值不需要是可序列化的，比如数据库连接、文件句柄或线程，此时无法用cache_data，只能用cache_resource。

* classwork 1

  1. 实现一个文件上传并输出
 
  2. 实现获得上传文件的目录并通过TextLoader导入，然后实现retriever检索对象的创建
 
  3. 定义函数，通过st.cache_resource装饰器把retriever检索对象放入缓存

## RAG应用实现

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
from langchain.vectorstores import Chroma
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

model_qf = QianfanLLMEndpoint(temperature=0.1)
loader_txt = TextLoader(r'G:\云岚宗.txt', encoding='utf8')
docs_txt = loader_txt.load()
text_splitter_txt = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["\n\n", "\n", " ", "", "。", "，"])
documents_txt = text_splitter_txt.split_documents(docs_txt)
embeddings_qf = QianfanEmbeddingsEndpoint()
vectordb = Chroma.from_documents(documents=documents_txt, embedding=embeddings_qf)

prompt = ChatPromptTemplate.from_template("""使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
        为了保证答案尽可能简洁，你的回答必须不超过三句话，你的回答中不可以带有星号。
        请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
        以下是一对问题和答案的样例：
            请问：秦始皇的原名是什么
            秦始皇原名嬴政。感谢你的提问。
        
        以下是语料：
<context>
{context}
</context>

Question: {input}""")
#创建检索链

retriever = vectordb.as_retriever()
memory = ConversationSummaryMemory(
    llm=model_qf, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm=model_qf, retriever=retriever, memory=memory)

[INFO] [05-21 09:34:45] openapi_requestor.py:316 [t:11484]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [05-21 09:34:45] oauth.py:207 [t:11484]: trying to refresh access_token for ak `og6mWr***`
[INFO] [05-21 09:34:45] oauth.py:220 [t:11484]: sucessfully refresh access_token


In [9]:
memory.chat_memory.messages

[]

In [5]:
res = qa.invoke(
    {"question": "萧炎是谁"}
)
print(res["answer"])

[WARNING] [05-21 09:34:55] base.py:495 [t:11484]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-21 09:34:55] openapi_requestor.py:316 [t:11484]: requesting llm api endpoint: /chat/eb-instant
[INFO] [05-21 09:34:56] openapi_requestor.py:316 [t:11484]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [05-21 09:34:56] base.py:495 [t:11484]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-21 09:34:56] openapi_requestor.py:316 [t:11484]: requesting llm api endpoint: /chat/eb-instant
[WARNING] [05-21 09:34:57] base.py:495 [t:11484]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-21 09:34:57] openapi_requestor.py:316 [t:11484]: requesting llm api endpoint: /chat/eb-instant


根据提供的上下文，问题"萧炎是谁"的目的是询问萧炎的身份或背景信息。可能的原因包括对故事中的人物感兴趣，或者需要了解更多关于这个角色的信息以便更好地理解故事情节。


In [6]:
res = qa.invoke(
    {"question": "他结婚了吗？"}
)
print(res["answer"])

[WARNING] [05-21 09:35:06] base.py:495 [t:11484]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-21 09:35:06] openapi_requestor.py:316 [t:11484]: requesting llm api endpoint: /chat/eb-instant
[INFO] [05-21 09:35:07] openapi_requestor.py:316 [t:11484]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [05-21 09:35:07] base.py:495 [t:11484]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-21 09:35:07] openapi_requestor.py:316 [t:11484]: requesting llm api endpoint: /chat/eb-instant
[WARNING] [05-21 09:35:08] base.py:495 [t:11484]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [05-21 09:35:08] openapi_requestor.py:316 [t:11484]: requesting llm api endpoint: /chat/eb-instant


根据原文信息可知，萧炎并没有结婚。文中提到纳兰嫣然是萧炎的未婚妻，但并没有提到他们已经结婚。萧炎和纳兰嫣然的婚约是家族之间的约定，但并没有实现。因此，可以得出结论，萧炎目前并没有结婚。


In [14]:
#help(QianfanEmbeddingsEndpoint)

## 基于LCEL实现自动查询 SQL 数据库

* sqlite数据库下载

链接：https://pan.baidu.com/s/1_nvpBZcrZ_3dgYD1HKX3rg?pwd=kp0h 
提取码：kp0h

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [5]:
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///ali_langchain.db")

def get_schema(x):
    return db.get_table_info()
def run_query(query):
    return db.run(query)
def get_sql(x):
    return x.split("```sql")[1].split("```")[0]

template ="""根据下面的表格结构，编写一个 SQL 查询来回答用户的问题：
{schema}

问题：{question}
给出的SQL查询应有如下形式:
```sql
...
```"""
prompt = ChatPromptTemplate.from_template(template)

chain_sql=({"schema":RunnableLambda(get_schema),"question":RunnablePassthrough()}|prompt|ChatTongyi()|StrOutputParser()
           |RunnableLambda(get_sql))

template_1 ="""根据下面的表格结构、问题、SQL 查询和 SQL 响应，编写一个自然语言回应：
{schema}

问题：{question}
SQL 查询：{query}
SQL 响应：{response}"""
prompt_response = ChatPromptTemplate.from_template(template_1)

chain_sql2=({"question":RunnablePassthrough(),"query":chain_sql}|RunnablePassthrough.assign(schema=get_schema,response=lambda x: run_query(x["query"]))|prompt_response|ChatTongyi())

In [10]:
db.get_table_info()

'\nCREATE TABLE alidata (\n\t"index" INTEGER, \n\tuser INTEGER, \n\tbrand INTEGER, \n\tbehavr INTEGER, \n\tdate TEXT\n)\n\n/*\n3 rows from alidata table:\nindex\tuser\tbrand\tbehavr\tdate\n0\t10944750\t13451\t0\t06/04\n1\t10944750\t13451\t2\t06/04\n2\t10944750\t13451\t2\t06/04\n*/'

In [6]:
chain_sql2.invoke("表里有多少brand？")

AIMessage(content='在这个alidata表中，有9531个不同的品牌。', response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '8644bd1e-0ceb-9a94-bf66-176e57655bc0', 'token_usage': {'input_tokens': 188, 'output_tokens': 15, 'total_tokens': 203}}, id='run-e3e8cb73-b676-48ca-b4a2-93fd28804d39-0')

In [17]:
chain_sql.invoke("表里所有brand的记录数,然后对这些记录数取平均值")

'\nSELECT brand, COUNT(*) AS record_count\nFROM alidata\nGROUP BY brand\nORDER BY brand\n'